In [2]:
import nbformat

def extract_cells(notebook_path):
    """ extracts the cells from the provided notebook. """
    with open(notebook_path, 'r', encoding='utf-8') as notebook_file:
        notebook_content = nbformat.read(notebook_file, as_version=4)

    cells = notebook_content['cells']
    return cells

# Example usage
notebook_path = '../BatchGenerateOutputs_2023-11-10.ipynb'
extracted_cells = extract_cells(notebook_path)
print(extracted_cells)


[{'cell_type': 'code', 'execution_count': 4, 'id': '0056bc66-7629-4ef7-8c87-f28f8fcd9dc8', 'metadata': {'autorun': True, 'notebookRunGroups': {'groupValue': '1'}, 'tags': ['imports', 'REQUIRED', 'ACTIVE']}, 'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': 'Automatic pdb calling has been turned OFF\nThe autoreload extension is already loaded. To reload it, use:\n  %reload_ext autoreload\n'}], 'source': "%config IPCompleter.use_jedi = False\n%pdb off\n%load_ext autoreload\n%autoreload 3\nimport sys\nfrom copy import deepcopy\nfrom typing import List, Dict, Optional, Union, Callable\nfrom pathlib import Path\nimport pathlib\nimport numpy as np\nimport pandas as pd\nimport tables as tb\nfrom copy import deepcopy\nfrom datetime import datetime, timedelta\nfrom attrs import define, field, Factory\n\n# required to enable non-blocking interaction:\n%gui qt5\n\n## Pho's Custom Libraries:\nfrom pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path\nfrom pyphocoreh

In [7]:
cells_with_tags = []
for cell in extracted_cells:
	cell_content = cell['source']
	cell_tags = cell['metadata'].get('tags', None)
	if (cell_tags is not None) and (len(cell_tags) > 0):
		cells_with_tags.append({'content': cell_content, 'tags': cell_tags})

cells_with_tags

[{'content': "%config IPCompleter.use_jedi = False\n%pdb off\n%load_ext autoreload\n%autoreload 3\nimport sys\nfrom copy import deepcopy\nfrom typing import List, Dict, Optional, Union, Callable\nfrom pathlib import Path\nimport pathlib\nimport numpy as np\nimport pandas as pd\nimport tables as tb\nfrom copy import deepcopy\nfrom datetime import datetime, timedelta\nfrom attrs import define, field, Factory\n\n# required to enable non-blocking interaction:\n%gui qt5\n\n## Pho's Custom Libraries:\nfrom pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path\nfrom pyphocorehelpers.function_helpers import function_attributes\nfrom pyphocorehelpers.print_helpers import CapturedException\n\n# Jupyter interactivity:\nimport ipywidgets as widgets\nfrom IPython.display import display\nfrom pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget\n\n# pyPhoPlaceCellAnalysis:\n# NeuroPy (Diba Lab Python Repo) Loading\nfrom neuropy.core.session.Formats.BaseD